<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_FINETUNE2_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
#install unsloth, xformers (for flash attn) and other pckgs
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [3]:
from unsloth import FastLanguageModel
import pandas as pd
import torch
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

# Constants
MODEL_NAME = "unsloth/mistral-7b-v0.3"
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True
BATCH_SIZE = 2
GRAD_ACCUMULATION = 4
LEARNING_RATE = 2e-4
TRAIN_STEPS = 100  # Optimized training steps
OUTPUT_DIR = "outputs"
SEED = 3407

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=LOAD_IN_4BIT
)

# Apply LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.1,  # Small dropout for regularization
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
)

# Load and process dataset
dataset = pd.read_csv("SOCOTEC_DATASET.csv")
EOS_TOKEN = tokenizer.eos_token

def format_data(examples):
    prompt = examples["Prompt"]
    function_call = examples["Function Call"]
    return {"text": f"{prompt}\n{function_call}{EOS_TOKEN}"}

# Apply formatting and convert to Hugging Face Dataset
dataset = Dataset.from_pandas(dataset)
dataset = dataset.map(format_data)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION,
    warmup_steps=5,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=SEED,
    output_dir=OUTPUT_DIR,
    report_to="none",
)

# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

# Train model
trainer_stats = trainer.train()

# Print memory usage and training time
gpu_stats = torch.cuda.get_device_properties(0)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"GPU: {gpu_stats.name}")
print(f"Peak memory usage: {used_memory} GB")
print(f"Training runtime: {trainer_stats.metrics['train_runtime']} seconds")

# Inference
FastLanguageModel.for_inference(model)
input_text = "What is 2 plus 3?"
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=16, use_cache=True)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(f"Generated Function Call: {result}")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Mistral patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

...

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/136 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 136 | Num Epochs = 6
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.551200
20,1.192500
30,0.997800
40,0.799900
50,0.763900
60,0.637700
70,0.655400
80,0.550700
90,0.496500
100,0.494200


GPU: NVIDIA A100-SXM4-40GB
Peak memory usage: 5.111 GB
Training runtime: 222.9281 seconds
Generated Function Call: What is 2 plus 3?
add(2, 3)


In [14]:
# Inference
FastLanguageModel.for_inference(model)
input_text = "increase 6 by 2"
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=16, use_cache=True)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(f"Generated Function Call: {result}")


Generated Function Call: increase 6 by 2
add(6, 2)


In [15]:
# Inference
FastLanguageModel.for_inference(model)
input_text = "say wassup to pete"
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=16, use_cache=True)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(f"Generated Function Call: {result}")


Generated Function Call: say wassup to pete
greet("Pete")


In [16]:
# Inference
FastLanguageModel.for_inference(model)
input_text = "increase 3 by negative 2"
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=16, use_cache=True)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(f"Generated Function Call: {result}")


Generated Function Call: increase 3 by negative 2
add(3, -2)


In [20]:
# Inference
FastLanguageModel.for_inference(model)
input_text = "cube two"
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=16, use_cache=True)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(f"Generated Function Call: {result}")


Generated Function Call: cube two numbers
cube(2)
